<a href="https://colab.research.google.com/github/xmxhuihui/MLiS2/blob/master/Object_Detection_ipynb(final)(epoch%3D16000).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

import tensorflow as tf
print(tf.__version__)
import os

TensorFlow 1.x selected.
1.15.2


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Number of training steps.
num_steps = 1000  # 200000
#num_steps = 100  # 200000

# Number of evaluation steps.
num_eval_steps = 50
# model configs are from Model Zoo github: 
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models
MODELS_CONFIG = {
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
    'ssd_mobilenet_v1_quantized': {
        'model_name': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18',
        'pipeline_file': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync.config',
        'batch_size': 12
    },    
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
    'ssd_mobilenet_v2_quantized': {
        'model_name': 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03',
        'pipeline_file': 'ssd_mobilenet_v2_quantized_300x300_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 12
    }
}
selected_model = 'ssd_mobilenet_v2_quantized'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [0]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
!pip install -U -q numpy==1.17.0
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py
    

/content
Selecting previously unselected package python-bs4.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpa

In [0]:
# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
%cd /content/drive/My Drive/Colab Notebooks
!python code/xml_to_csv.py -i Object -o annotations/train_labels.csv -l annotations

# Convert test folder annotation xml files to a single csv.
!python code/xml_to_csv.py -i Object -o annotations/test_labels.csv

# Generate `train.record`
!python code/generate_tfrecord.py --csv_input=annotations/train_labels.csv --output_path=annotations/train.record --img_path=Object --label_map annotations/label_map.pbtxt

# Generate `test.record`
!python code/generate_tfrecord.py --csv_input=annotations/test_labels.csv --output_path=annotations/test.record --img_path=Object --label_map annotations/label_map.pbtxt

/content/drive/My Drive/Colab Notebooks
Successfully converted xml to csv.
Generate `annotations/label_map.pbtxt`
Successfully converted xml to csv.


W0503 16:18:01.943094 140070056109952 module_wrapper.py:139] From code/generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0503 16:18:02.609474 140070056109952 module_wrapper.py:139] From /content/models/research/object_detection/utils/label_map_util.py:138: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/drive/My Drive/Colab Notebooks/annotations/train.record


W0503 16:19:03.208383 140397043120000 module_wrapper.py:139] From code/generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0503 16:19:03.944317 140397043120000 module_wrapper.py:139] From /content/models/research/object_detection/utils/label_map_util.py:138: The name t

In [0]:
test_record_fname = '/content/drive/My Drive/Colab Notebooks/annotations/test.record'
train_record_fname = '/content/drive/My Drive/Colab Notebooks/annotations/train.record'
label_map_pbtxt_fname = '/content/drive/My Drive/Colab Notebooks/annotations/label_map.pbtxt'
print(test_record_fname)
print(train_record_fname)
print(label_map_pbtxt_fname)

/content/drive/My Drive/Colab Notebooks/annotations/test.record
/content/drive/My Drive/Colab Notebooks/annotations/train.record
/content/drive/My Drive/Colab Notebooks/annotations/label_map.pbtxt


In [0]:
!cat annotations/train_labels.csv

filename,width,height,class,xmin,ymin,xmax,ymax
1582722545700_95_0.png,320,240,large female,179,68,238,223
1584031891105_90_35.png,320,240,small male,108,37,125,70
1584031891105_90_35.png,320,240,red light,122,10,130,32
1584031891105_90_35.png,320,240,left sign,135,16,143,32
1584029456776_90_0.png,320,240,small male,160,69,187,139
1584031273877_90_35.png,320,240,green light,129,10,138,42
1584031273877_90_35.png,320,240,left sign,147,17,160,44
1584029455744_90_0.png,320,240,small male,161,69,187,140
1582723269156_85_0.png,320,240,large male,111,62,164,236
1582723269156_85_0.png,320,240,large female,63,61,109,217
1584031274909_90_35.png,320,240,green light,126,13,139,51
1584031274909_90_35.png,320,240,left sign,146,21,161,53
1584031896275_90_0.png,320,240,small male,61,85,100,182
1584031896275_90_0.png,320,240,red light,122,10,132,34
1584031896275_90_0.png,320,240,left sign,138,17,149,35
1584031896275_90_0.png,320,240,small female,55,12,65,38
1582727151921_95_35.png,320,240,large male,12

In [0]:
!cat annotations/test_labels.csv

filename,width,height,class,xmin,ymin,xmax,ymax
1582722545700_95_0.png,320,240,large female,179,68,238,223
1584031891105_90_35.png,320,240,small male,108,37,125,70
1584031891105_90_35.png,320,240,red light,122,10,130,32
1584031891105_90_35.png,320,240,left sign,135,16,143,32
1584029456776_90_0.png,320,240,small male,160,69,187,139
1584031273877_90_35.png,320,240,green light,129,10,138,42
1584031273877_90_35.png,320,240,left sign,147,17,160,44
1584029455744_90_0.png,320,240,small male,161,69,187,140
1582723269156_85_0.png,320,240,large male,111,62,164,236
1582723269156_85_0.png,320,240,large female,63,61,109,217
1584031274909_90_35.png,320,240,green light,126,13,139,51
1584031274909_90_35.png,320,240,left sign,146,21,161,53
1584031896275_90_0.png,320,240,small male,61,85,100,182
1584031896275_90_0.png,320,240,red light,122,10,132,34
1584031896275_90_0.png,320,240,left sign,138,17,149,35
1584031896275_90_0.png,320,240,small female,55,12,65,38
1582727151921_95_35.png,320,240,large male,12

## Download Base Model

In [0]:
import os
import shutil
import glob
import urllib.request
import tarfile

%cd /content/models/research
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [0]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/models/research/pretrained_model
total 204M
drwx------  2 303230 5000 4.0K Jan  4  2019 .
drwxr-xr-x 63 root   root 4.0K May  3 16:19 ..
-rw-------  1 303230 5000  93M Jan  4  2019 model.ckpt.data-00000-of-00001
-rw-------  1 303230 5000  68K Jan  4  2019 model.ckpt.index
-rw-------  1 303230 5000  20M Jan  4  2019 model.ckpt.meta
-rw-------  1 303230 5000 4.3K Jan  4  2019 pipeline.config
-rw-------  1 303230 5000  24M Jan  4  2019 tflite_graph.pb
-rw-------  1 303230 5000  68M Jan  4  2019 tflite_graph.pbtxt


In [0]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/models/research/pretrained_model/model.ckpt'

# Transfer Learning Training

In [0]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [0]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [0]:
import re  #regular expression

# training pipeline file defines:
# - pretrain model path
# - the train/test sets
# - ID to Label mapping and number of classes
# - training batch size
# - epochs to trains
# - learning rate
# - etc

# note we just need to use a sample one, and make edits to it.

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint: downloaded pre-trained model checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test, we created earlier with our training/test sets
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path: ID to label file
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps (Number of epochs to train)
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [0]:
!cat "{label_map_pbtxt_fname}"

item {
    id: 1
    name: 'box'
}

item {
    id: 2
    name: 'green light'
}

item {
    id: 3
    name: 'large female'
}

item {
    id: 4
    name: 'large male'
}

item {
    id: 5
    name: 'left sign'
}

item {
    id: 6
    name: 'red light'
}

item {
    id: 7
    name: 'right sign'
}

item {
    id: 8
    name: 'small female'
}

item {
    id: 9
    name: 'small male'
}

item {
    id: 10
    name: 'tree'
}

In [0]:
!cat {pipeline_fname}

# Quantized trained SSD with Mobilenet v2 on MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 10
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        asp

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
model_dir = '/content/drive/My Drive/Colab Notebooks/Training'
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir "{}" --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-05-03 16:21:54--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.226.171.201, 3.221.253.104, 3.228.72.85, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.226.171.201|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.13M  6.49MB/s    in 2.0s    

2020-05-03 16:21:57 (6.49 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://1e8c0396.ngrok.io


In [0]:
num_steps = 16000

!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir='{model_dir}' \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}
print("Training finished.")

# Save and Convert Model Output

In [0]:
import os
import re
import numpy as np

output_directory = '%s/fine_tuned_model' % model_dir
os.makedirs(output_directory, exist_ok=True)

In [0]:
lst = os.listdir(model_dir)
# find the last model checkpoint file, i.e. model.ckpt-1000.meta
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

In [0]:
!echo creates the frozen inference graph in fine_tune_model
# there is an "Incomplete shape" message.  but we can safely ignore that. 
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory='{output_directory}' \
    --trained_checkpoint_prefix='{last_model_path}'

In [0]:
# https://medium.com/tensorflow/training-and-serving-a-realtime-mobile-object-detector-in-30-minutes-with-cloud-tpus-b78971cf1193
# create the tensorflow lite graph
#!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
#    --pipeline_config_path={pipeline_fname} \
#    --trained_checkpoint_prefix='{last_model_path}' \
#    --output_directory='{output_directory}' \
#    --add_postprocessing_op=true

In [0]:
#!echo "CONVERTING frozen graph to quantized TF Lite file..."
#!tflite_convert \
#  --output_file='{output_directory}/road_signs_quantized.tflite' \
#  --graph_def_file='{output_directory}/tflite_graph.pb' \
#  --inference_type=QUANTIZED_UINT8 \
#  --input_arrays='normalized_input_image_tensor' \
#  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
#  --mean_values=128 \
#  --std_dev_values=128 \
#  --input_shapes=1,300,300,3 \
#  --change_concat_input_ranges=false \
#  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
#  --allow_custom_ops

In [0]:
#!echo "CONVERTING frozen graph to unquantized TF Lite file..."
#!tflite_convert \
#  --output_file='{output_directory}/road_signs_float.tflite' \
#  --graph_def_file='{output_directory}/tflite_graph.pb' \
# --input_arrays='normalized_input_image_tensor' \
#  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
#  --mean_values=128 \
#  --std_dev_values=128 \
#  --input_shapes=1,300,300,3 \
#  --change_concat_input_ranges=false \
#  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
# --allow_custom_ops

In [0]:
print(output_directory)
!ls -ltra '{output_directory}'
#pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb") # this is main one
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")  # this is tflite graph
!cp '{label_map_pbtxt_fname}' '{output_directory}'

##Run Inference Test

In [0]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname
print(PATH_TO_CKPT)

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
#PATH_TO_TEST_IMAGES_DIR =  os.path.join(repo_dir_path, "models/object_detection/data/images/test")
PATH_TO_TEST_IMAGES_DIR='/content/drive/My Drive/Colab Notebooks/test_data'

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.png"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

In [0]:
%cd /content/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [0]:
def label_output_for_single_image(output_dict):
  score_threshold=0.5
  labels=[]
  for i in range(len(output_dict['detection_scores'])):
    if output_dict['detection_scores'][i]>score_threshold:
      if (output_dict['detection_boxes'][i][2]-output_dict['detection_boxes'][i][0])>0.05:
        labels.append(output_dict['detection_classes'][i])
    else:
      break
  return labels

In [0]:
## running inferences.  This should show images with bounding boxes
#%matplotlib inline
#
#print('Running inferences on %s' % TEST_IMAGE_PATHS)
#for image_path in TEST_IMAGE_PATHS:
#    image = Image.open(image_path)
#    # the array based representation of the image will be used later in order to prepare the
#    # result image with boxes and labels on it.
#    image_np = load_image_into_numpy_array(image)
#    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
#    image_np_expanded = np.expand_dims(image_np, axis=0)
#    # Actual detection.
#    output_dict = run_inference_for_single_image(image_np, detection_graph)
#    # Visualization of the results of a detection.
#    vis_util.visualize_boxes_and_labels_on_image_array(
#        image_np,
#        output_dict['detection_boxes'],
#        output_dict['detection_classes'],
#        output_dict['detection_scores'],
#        category_index,
#        instance_masks=output_dict.get('detection_masks'),
#        use_normalized_coordinates=True,
#        line_thickness=2)
#    plt.figure(figsize=IMAGE_SIZE)
#    plt.imshow(image_np)

In [0]:
# Too many figures would cause RuntimeError, so just plot a batch of figures.
# running inferences.  This should show images with bounding boxes
import re
%matplotlib inline

print('Running inferences on %s' % TEST_IMAGE_PATHS)
label_lists=[[] for i in range(len(TEST_IMAGE_PATHS))]
boxes_bottom_mid=[[] for i in range(len(TEST_IMAGE_PATHS))]
boxes_bottom=[[] for i in range(len(TEST_IMAGE_PATHS))]
for image_path in TEST_IMAGE_PATHS:
#image_path=TEST_IMAGE_PATHS[31]
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=2)
    #plt.figure(figsize=IMAGE_SIZE)
    #plt.imshow(image_np)
    image_index=int(re.findall(r'(\d+).png',image_path)[0])
    print(image_index)
    label_list=label_output_for_single_image(output_dict)
    label_lists[image_index-1]=label_list
    for i in range(len(output_dict['detection_classes'])):
      if output_dict['detection_scores'][i]>0.5 and output_dict['detection_classes'][i] in [1,10]:
        boxes_bottom_mid[image_index-1].append((output_dict['detection_boxes'][i][1]+output_dict['detection_boxes'][i][3])/2)
        boxes_bottom[image_index-1].append(output_dict['detection_boxes'][i][2])

print(label_lists)

print(boxes_bottom_mid)


In [0]:
import cv2
def location_detection(i,bottom_mid,bottom):
  path=os.path.join(PATH_TO_TEST_IMAGES_DIR,str(i+1)+'.png')
  img=cv2.imread(path)
  #plt.imshow(img)
  height, width, _ = img.shape
  img = img[int(height/2):,:,:]
  gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  mask = cv2.inRange(gray,0,10)
  ret, binary = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)
  kernel = np.ones((5, 5), np.uint8)
  dilation = cv2.dilate(binary, kernel, iterations=1)
  #plt.imshow(dilation)
  lines = cv2.HoughLinesP(
    binary,
    rho=1,
    theta=np.pi / 180,
    threshold=10,
    lines=np.array([]),
    minLineLength=8,
    maxLineGap=4
)
  
  #print(lines)
  for line in lines:
      for x1, y1, x2, y2 in line:
          cv2.line(img,(x1,y1),(x2,y2),((255, 0, 0)),2)
  #plt.imshow(img)
  
  def make_points(frame, line):
    height, width, _ = frame.shape
    slope, intercept = line
    intercept=intercept+height
    y1 = height*2  # bottom of the frame
    #y2 = int(y1 * 1 / 2)  # make points from middle of the frame down
    y2 = 0
    # bound the coordinates within the frame
    x1 = max(-width, min(2 * width, int((y1 - intercept) / slope)))
    x2 = max(-width, min(2 * width, int((y2 - intercept) / slope)))
    return [[x1, y1, x2, y2]]

  lane_lines = []
  height, width, _ = img.shape
  left_fit = []
  right_fit = []
  boundary = 1/3
  left_region_boundary = width * (1 - boundary)  # left lane line segment should be on left 2/3 of the screen
  right_region_boundary = width * boundary # right lane line segment should be on left 2/3 of the screen
  for line_segment in lines:
      for x1, y1, x2, y2 in line_segment:
        try:
          fit = np.polyfit((x1, x2), (y1, y2), 1)
          
          slope = fit[0]
          intercept = fit[1]
          if slope < 0:
              if x1 < left_region_boundary and x2 < left_region_boundary:
                  left_fit.append((slope, intercept))
          else:
              if x1 > right_region_boundary and x2 > right_region_boundary:
                  right_fit.append((slope, intercept))
        except:
          pass
  
  if len(left_fit) > 0:
      left_fit_average = np.average(left_fit, axis=0)
      lane_lines.append(make_points(img, left_fit_average))
  else:
      left_fit_average=[]
  
  if len(right_fit) > 0:
      right_fit_average = np.average(right_fit, axis=0)
      lane_lines.append(make_points(img, right_fit_average))
  else:
      right_fit_average=[]
  #print(lane_lines)
  image=cv2.imread(path)
  for line in lane_lines:
      for x1, y1, x2, y2 in line:
          cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
  plt.imshow(image)
  for i in range(len(bottom)):
    y=bottom[i]*height
    x=bottom_mid[i]*width
    if len(left_fit_average)!=0 and len(right_fit_average)==0:
      if y>left_fit_average[0]*x+left_fit_average[1]+height/2:
        return 0
    elif len(left_fit_average)==0 and len(right_fit_average)!=0:
      if y>right_fit_average[0]*x+right_fit_average[1]+height/2:
        return 0
    elif len(left_fit_average)!=0 and len(right_fit_average)!=0:
      if x<(y-right_fit_average[1]-height/2)/right_fit_average[0] and x>(y-left_fit_average[1]-height/2)/left_fit_average[0]:
        return 0
  return 1

In [0]:
stop_sign=[3,4,6,8,9]
judge_sign=[1,10]
norm_speed=np.ones(len(label_lists),dtype=np.int32)
for i in range(len(label_lists)):
  for j in range(len(label_lists[i])):
    if label_lists[i][j] in stop_sign:
      norm_speed[i]=0
      break
    if label_lists[i][j] in judge_sign:
      print(i)
      norm_speed[i]=location_detection(i,boxes_bottom_mid[i],boxes_bottom[i])
      if norm_speed[i]==0:
        break
print(norm_speed)

In [0]:
import pandas as pd
table=pd.DataFrame(norm_speed,index=list(range(1,len(label_lists)+1)),columns=['speed'])
table.head()

In [0]:
table['image_id']=list(range(1,len(label_lists)+1))
table=table.reindex(columns=['image_id','speed'])
table.head()

In [0]:
table.to_csv('/content/drive/My Drive/Colab Notebooks/speed.csv',index=False)